In [35]:
import pyaudio
import pylab
import os
from struct import pack
from sys import byteorder
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.signal import find_peaks
from scipy.fftpack import fft
import time
from tkinter import TclError
from scipy.io import wavfile as wav
import wave
from pydub import AudioSegment
from collections import Counter
from array import array
from pydub.silence import split_on_silence
import crepe







In [36]:
# White keys are in Uppercase and black keys (sharps) are in lowercase
octave = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'] 
base_freq = 440 #Frequency of Note A4
keys = np.array([x+str(y) for y in range(0,9) for x in octave])
# Trim to correct # keys
start = np.where(keys == 'A0')[0][0]
end = np.where(keys == 'C8')[0][0]
keys = keys[start:end+1]

note_freqs = list(([2**((n-48)/12)*base_freq for n in range(len(keys))]))

In [37]:
c_major_scale_freqs=[]
c_major_scale=['C4', 'D4', 'E4', 'F4', 'G4']
for i in range(len(keys)):
    for j in range(5):
        if c_major_scale[j]==keys[i]:
            c_major_scale_freqs.append(note_freqs[i])
print(c_major_scale_freqs)

[261.6255653005986, 293.6647679174076, 329.6275569128699, 349.2282314330039, 391.99543598174927]


In [38]:
def closest_value(input_list, input_value):

  arr = np.asarray(input_list)

  i = (np.abs(arr - input_value)).argmin()

  return arr[i]

In [39]:
# constants
CHUNK = 1024*4          # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 48000                 # samples per second
RECORD_SECONDS = 1

In [40]:
p = pyaudio.PyAudio()
p.get_device_info_by_index(3)['defaultSampleRate']

48000.0

In [30]:
#os.remove('/Users/emmarobinson/Downloads/piano_audio.wav') #change this according to your computer file
#os.remove('/Users/emmarobinson/Downloads/piano_audio_new.wav') #change this according to your computer file
# create matplotlib figure and axes
#fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance

p = pyaudio.PyAudio()
#p.get_device_info_by_index(3)['defaultSampleRate']

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    #input_device_index =3, #change this according to the mic port on your computer
    input=True,output=True,
    frames_per_buffer=CHUNK
)
frames = []
print("* listening")

while 1:
    
    data = stream.read(CHUNK, exception_on_overflow = False)
    data_chunk=array('h',data)
    vol=max(data_chunk)
    if vol >= 400:
        print("* recording")
        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            stream.write(data, CHUNK)
            frames.append(data)
        break

    else:
        print("* waiting")

print("* done")

stream.stop_stream()
stream.close()

p.terminate()
file_path='/Users/emmarobinson/Downloads/piano_audio.wav' #change this according to your computer file
file_name = file_path.split('/')[-1]
wf = wave.open(file_path, "wb")
# set the channels
wf.setnchannels(CHANNELS)
# set the sample format
wf.setsampwidth(p.get_sample_size(FORMAT))
# set the sample rate
wf.setframerate(RATE)
# write the frames as bytes
wf.writeframes(b"".join(frames))
# close the file
wf.close()


* listening
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* waiting
* recording
* done


In [27]:
# I dont know if we need this tbh
sound = AudioSegment.from_file(file_path, format = 'wav') 
audio_chunks = split_on_silence(sound
                            ,min_silence_len = 100
                            ,silence_thresh = -45
                            ,keep_silence = 50
                        )

# Putting the file back together
combined = AudioSegment.empty()
for chunk in audio_chunks:
    combined += chunk
combined.export(f'/Users/emmarobinson/Downloads/piano_audio_new.wav', format = 'wav') #change this according to your computer file

<_io.BufferedRandom name='/Users/emmarobinson/Downloads/piano_audio_new.wav'>

In [32]:
sr, audio = wav.read('/Users/emmarobinson/Downloads/piano_audio.wav') #change this according to your computer file
time, frequency, confidence, activation = crepe.predict(audio, sr, viterbi=True, model_capacity="small")
freq1=np.array([np.mean(frequency)])

3/3 [==============================] - 1s 378ms/step


In [33]:
song_notes=[]
for i in range(len(freq1)):
    note=closest_value(note_freqs,freq1[i])
    index = note_freqs.index(note)
    song_notes.append(keys[index])
    print(song_notes)

['C#6']


In [ ]:
for i in range(len(freq1)):
    note=closest_value(note_freqs,freq1[i])
    index = note_freqs.index(note)
    played_note=keys[index]

x=0
if played_note==c_major_scale[x]:
    print("yay")
else:
        print("try again")

In [155]:
rate, data = wav.read('/Users/emmarobinson/Downloads/piano_audio_new.wav') #change this according to your computer file

audio_segment = AudioSegment.from_file('/Users/emmarobinson/Downloads/piano_audio_new.wav') #change this according to your computer file

duration = len(audio_segment)/1000

# calculate the length of our chunk in the np.array using sample rate
chunk = int(rate * duration)

# length of delimiting 1600hz tone
offset = int(rate * 0.005)

# number of bits in the audio data to decode
bits = int(len(data) / chunk)

def get_freq(bit):
    # start position of the current bit
    strt = (chunk * bit) 
    
    # remove the delimiting 1600hz tone
    end = (strt + chunk) 
   
    
    # slice the array for each bit
    sliced = data[strt:end]

    w = np.fft.fft(sliced)
    w=np.absolute(w)
    freqs = np.fft.fftfreq(len(w))

    

    # Find the peak in the coefficients
    idx = np.argmax(np.abs(w))
    freq = freqs[idx]
    freq_in_hertz = abs(freq * rate)
    return freq_in_hertz

decoded_freqs = [get_freq(bit) for bit in range(bits)]